# Código principal

Nesse código é implementado todo o pipeline proposto pelo projeto, como definido na imagem a seguir:

![image](../diagramaPI.drawio.png)

## Instalação das bibliotecas necessárias

In [ ]:
%pip install numpy
%pip install pandas
%pip install pytesseract
%pip install opencv-python
%pip install matplotlib
%pip install imutils
%pip install ipywidgets
%pip install scikit-learn

## Importações de bibliotecas externas

In [ ]:
import pytesseract
import ipywidgets as widgets
import cv2
import asyncio

Para configurar o `pytesseract` é preciso instalar o seu executável. Se não tiver configurado ele nas variáveis de ambiente, modificar a variável do código abaixo para indicar o caminho até o executável do `tesseract`.

A documentação do `pytesseract` pode ser [acessada aqui](https://pypi.org/project/pytesseract/)

O `tesseract` pode ser instalado [por aqui](https://tesseract-ocr.github.io/tessdoc/Installation.html)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Importações internas

In [ ]:
from image_preprocessing.image_preprocessing import preprocess_image
from image_preprocessing.document_digitalization import mark_document_found
from image_preprocessing.rotations import rotate_180

from OCR.image_to_dataframe import image_to_dataframe

from relevant_info_identification.relevant_info_identification import get_document_info_and_mark_relevant_regions

## Criando objetos da interface gráfica do projeto

In [ ]:
%%html
<style>
.selected{
    background-color: rgb(0, 255, 0);
}
.not-selected{
    background-color: rgb(255, 0, 0);
}
</style>

In [ ]:
original_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)


process_btn = widgets.Button(
    description='Processar',
    button_style='success',
    icon='check'
)

stop_btn = widgets.Button(
    description='Parar',
    button_style='danger',
    icon='times'
)


no_rotate_marked_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)

no_rotate_infos_json_widget = widgets.Text()

no_rotate_prob_widget = widgets.Text()


rotate_180_marked_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)

rotate_180_infos_json_widget = widgets.Text()

rotate_180_prob_widget = widgets.Text()


no_rotate_box = widgets.VBox([
    no_rotate_marked_image_widget,
    no_rotate_infos_json_widget,
    no_rotate_prob_widget
])
no_rotate_box.add_class('selected')


rotate_180_box = widgets.VBox([
    rotate_180_marked_image_widget,
    rotate_180_infos_json_widget,
    rotate_180_prob_widget
])
rotate_180_box.add_class('not-selected')
pass

## Definindo função do pipeline completo

In [ ]:
original_image = None

def execute_pipeline():
    if original_image is not None:
        preprocessed_image = preprocess_image(original_image)

        no_rotate_preprocessed_image = preprocessed_image
        rotate_180_preprocessed_image = rotate_180(preprocessed_image)

        no_rotate_relevant_info, no_rotate_marked_img = get_document_info_and_mark_relevant_regions(no_rotate_preprocessed_image)
        rotate_180_relevant_info, rotate_180_marked_img = get_document_info_and_mark_relevant_regions(rotate_180_preprocessed_image)

        # TODO COLOCAR CALCULO DE PROBABILIDADES

        no_rotate_marked_image_widget.value = cv2.imencode('.jpg', no_rotate_marked_img)[1].tobytes()
        no_rotate_infos_json_widget.valeu = no_rotate_relevant_info
        # TODO COLOCAR PROB

        rotate_180_marked_image_widget.value = cv2.imencode('.jpg', rotate_180_marked_img)[1].tobytes()
        rotate_180_infos_json_widget.valeu = rotate_180_relevant_info
        # TODO COLOCAR PROB

        # TODO MARCAR O MELHOR CONJUNTO DE RESULTADOS COLETADOS

delay_in_seconds = 0.1
camera = cv2.VideoCapture(0)
async def read_image_real_time():
    global original_image
    while True:
        ret, original_image = camera.read()
        frame_copy = original_image.copy()
        marked_image = mark_document_found(frame_copy)
        original_image_widget.value = cv2.imencode('.jpg', marked_image)[1].tobytes()
        await asyncio.sleep(delay_in_seconds)

loop = asyncio.get_event_loop()
task = loop.create_task(read_image_real_time())

def on_stop_btn_click():
    task.cancel()
    camera.release()

stop_btn.on_click(lambda _: on_stop_btn_click())


## Adicionando eventos e criando loop de captura de tela

In [ ]:
delay_in_seconds = 0.1
camera = cv2.VideoCapture(0)
async def read_image_real_time():
    global original_image
    while True:
        ret, original_image = camera.read()
        frame_copy = original_image.copy()
        marked_image = mark_document_found(frame_copy)
        original_image_widget.value = cv2.imencode('.jpg', marked_image)[1].tobytes()
        await asyncio.sleep(delay_in_seconds)

loop = asyncio.get_event_loop()
task = loop.create_task(read_image_real_time())

def on_stop_btn_click():
    task.cancel()
    camera.release()

process_btn.on_click(lambda _: execute_pipeline())
stop_btn.on_click(lambda _: on_stop_btn_click())

## Mostrando interface

In [ ]:
display(original_image_widget)

display(widgets.HBox([
    process_btn,
    stop_btn
]))

display(widgets.HBox([
    no_rotate_box,
    rotate_180_box
]))